```{contents}
```
## Output Validation 


**Output validation** is the process of **verifying, filtering, and correcting** the response generated by a system (especially an LLM) **before it is returned to the user**.

It ensures the output is:

* Safe
* Correctly formatted
* Policy-compliant
* Free of forbidden or dangerous content

---

### Where It Fits in the Pipeline

```
User Request
   ↓
LLM Generates Output
   ↓
Output Validator ── valid → Return to User
   ↓ invalid
Fix / Block / Regenerate / Log
```

---

### Why Output Validation Matters

| Risk              | Consequence           |
| ----------------- | --------------------- |
| Hallucinations    | Wrong answers         |
| Policy violations | Legal / safety issues |
| Bad formatting    | System failure        |
| Data leakage      | Security breach       |

---

### Schema & Format Validation

#### Demonstration

```python
from pydantic import BaseModel, ValidationError

class LLMResponse(BaseModel):
    answer: str
    confidence: float

def validate_format(raw_output):
    try:
        return LLMResponse.parse_raw(raw_output)
    except ValidationError:
        return None
```

---

### Content Safety Validation

#### Demonstration

```python
BLOCKED_TERMS = ["password", "secret", "credit card"]

def contains_blocked_content(text):
    for term in BLOCKED_TERMS:
        if term in text.lower():
            return True
    return False
```

---

### Length & Constraint Validation

#### Demonstration

```python
MAX_TOKENS = 500

def validate_constraints(text):
    if len(text.split()) > MAX_TOKENS:
        return False
    return True
```

---

### End-to-End Validation Pipeline

#### Demonstration

```python
def validate_output(raw_output):
    if not validate_constraints(raw_output):
        return None

    if contains_blocked_content(raw_output):
        return None

    return raw_output
```

---

### Enforcement & Recovery Strategy

```python
def safe_generate(prompt):
    response = llm.invoke(prompt).content

    valid = validate_output(response)
    if not valid:
        return "Response blocked: output failed validation."

    return valid
```

---

### Mental Model

```
Output Validation = Final quality gate before the user sees anything
```

---

### Key Takeaways

* Never trust model output blindly
* Validate format, content, and constraints
* Combine with input sanitization and injection detection
* Mandatory for production AI systems
